In [ ]:
# Add project src to path.
import set_path

import logging
import pandas as pd
import numpy as np

from src.utils.labels import Labels
import src.analysis.analysis_tools as analysis_tools
import src.utils.ahn_utils as ahn_utils
import src.utils.bgt_utils as bgt_utils
import src.utils.csv_utils as csv_utils
import src.utils.las_utils as las_utils
import src.utils.log_utils as log_utils

In [ ]:
# Set-up logging.
log_utils.reset_logger()
log_utils.add_console_logger(level=logging.ERROR)
# log_utils.add_console_logger(level=logging.DEBUG)
#log_utils.add_file_logger('weesp_merge_log.txt', level=logging.DEBUG, clear_log=True)

## Demo tiles

In [ ]:
# AHN data folder.
ahn_data_folder = '../datasets/ahn/'
ahn_reader = ahn_utils.NPZReader(ahn_data_folder, caching=False)

bgt_building_file = '../datasets/bgt/bgt_buildings_demo.csv'
bld_reader = bgt_utils.BGTPolyReader(bgt_building_file, building_offset=0)

In [ ]:
# Original pointcloud data folder.
cloud_folder = '../datasets/pointcloud/'
cloud_prefix = 'processed'
# Predicted pointcloud labels data folder.
pred_folder = '../datasets/pointcloud/'
pred_prefix = 'processed'

# File to write pole location to.
pole_locations_path = '../datasets/pointcloud/lamp_post_locations.csv'

In [ ]:
min_component_size = 100
target_label = Labels.STREET_LIGHT
ground_label = Labels.GROUND

HEADERS = ['rd_x', 'rd_y', 'z', 'tx', 'ty', 'tz', 'height', 'angle', 'prob', 'n_points', 'in_bld', 'debug', 'tilecode']

locations = analysis_tools.get_pole_locations_pred(cloud_folder, pred_folder, target_label, ground_label, 
                                                   ahn_reader=ahn_reader, bld_reader=bld_reader,
                                                   cloud_prefix=cloud_prefix, pred_prefix=pred_prefix,
                                                   min_component_size=min_component_size)

In [ ]:
csv_utils.write_csv(pole_locations_path, locations, csv_headers=HEADERS)

## Weesp

In [ ]:
# AHN data folder.
ahn_data_folder = '../../datasets/Weesp/ahn_tiles/'
ahn_reader = ahn_utils.NPZReader(ahn_data_folder, caching=False)

bgt_building_file = '../../datasets/Weesp/bgt/bgt_buildings_weesp.csv'
bld_reader = bgt_utils.BGTPolyReader(bgt_building_file, building_offset=0)

In [ ]:
# Original pointcloud data folder.
orig_folder = '../../datasets/Weesp_test/'
orig_prefix = 'merged'
# Predicted pointcloud labels data folder.
pred_folder = '../../datasets/Weesp_test/'
pred_prefix = 'merged'
# File to write pole location to.
pole_locations_path = 'lamp_posts_weesp_test.csv'

In [ ]:
# Original pointcloud data folder.
orig_folder = '../../datasets/Weesp/las_processor_bundled_out/'
orig_prefix = 'filtered'
# Predicted pointcloud labels data folder.
pred_folder = '../../datasets/Weesp/pred_rgb/'
pred_prefix = 'filtered'
# File to write pole location to.
pole_locations_path = 'lamp_posts_weesp_complete_rgb.csv'

In [ ]:
# Original pointcloud data folder.
orig_folder = '../../datasets/Weesp/predicted_rgbi/'
orig_prefix = 'merged'
# Predicted pointcloud labels data folder.
pred_folder = orig_folder
pred_prefix = orig_prefix
# File to write pole location to.
pole_locations_path = 'lamp_posts_weesp_rgbi_100.csv'

In [ ]:
min_component_size = 100
target_label = Labels.STREET_LIGHT
ground_label = Labels.GROUND
tilecodes = None
# tilecodes = ['2605_9578']

HEADERS = ['rd_x', 'rd_y', 'z', 'tx', 'ty', 'tz', 'height', 'angle', 'prob', 'n_points', 'in_bld', 'debug', 'tilecode']

locations = analysis_tools.get_pole_locations_pred(orig_folder, pred_folder, target_label, ground_label,
                                                   tilecodes=tilecodes, ahn_reader=ahn_reader, bld_reader=bld_reader,
                                                   cloud_prefix=orig_prefix, pred_prefix=pred_prefix,
                                                   min_component_size=min_component_size)

In [ ]:
pole_locations_path = 'lamp_posts_weesp_rgbi_100_new4.csv'

In [ ]:
csv_utils.write_csv(pole_locations_path, locations, csv_headers=HEADERS)

In [ ]:
df = pd.DataFrame(locations, columns=HEADERS)

In [ ]:
df.sort_values(by='angle', ascending=False).head(n=25)

## Extract BGT tiles

In [ ]:
df_bgt = pd.read_csv('bgt_weesp.csv', delimiter=',')

In [ ]:
import glob

las_tiles = glob.glob('../../datasets/Weesp/predicted_rgbi/*.laz')
df_bgt_tiles = pd.DataFrame(columns=['Type', 'X', 'Y'])

for tile in las_tiles:
    ((bx_min, by_max), (bx_max, by_min)) = las_utils.get_bbox_from_las_file(tile)
    df_tile = df_bgt.query('(X < @bx_max) & (X >= @bx_min)' +
                           ' & (Y <= @by_max) & (Y > @by_min)')
    df_bgt_tiles = pd.concat([df_bgt_tiles, df_tile])

In [ ]:
df_bgt_tiles.to_csv('bgt_tiles_weesp.csv')

## Match BGT with extracted poles

In [ ]:
df_bgt = pd.read_csv('bgt_tiles_weesp.csv', delimiter=',')

In [ ]:
df_bgt = df_bgt[df_bgt.Type == 'lichtmast']

In [ ]:
df_ext = pd.read_csv('lamp_posts_weesp_rgbi_100_new4.csv', delimiter=',')

In [ ]:
df_ext = df_ext[df_ext.height >= 3.]
df_ext = df_ext[df_ext.in_bld == 0]

In [ ]:
df_ext[df_ext.debug.str.endswith('3')]

In [ ]:
df_ext[df_ext.in_bld == 1]

In [ ]:
from scipy.spatial.distance import cdist
dist = cdist(df_bgt[['X', 'Y']].values, df_ext[['rd_x', 'rd_y']].values, metric='euclidean')

In [ ]:
dist.shape

In [ ]:
ext_min = np.min(dist, axis=0)
bgt_min = np.min(dist, axis=1)

In [ ]:
np.count_nonzero(ext_min < 0.1)

In [ ]:
np.count_nonzero(ext_min < 0.5)

In [ ]:
df_ext['dist_lichtmast'] = np.around(ext_min, 2)

In [ ]:
df_ext.to_csv('lichtmasten_weesp_extracted.csv')

## Analysis of extracted lamp posts

In [ ]:
df = pd.read_csv('lamp_posts_weesp_rgbi_100_new4.csv', delimiter=',')

In [ ]:
df[df.tilecode == '2620_9581']

In [ ]:
df = df[(df.height >= 3) & (df.height <= 20)]

In [ ]:
df.groupby(by='tilecode').size().sort_values().tail(n=20)

In [ ]:
len(df)

In [ ]:
len(df[df.n_points >= 1000])

In [ ]:
len(df[(df.height < 3)])

In [ ]:
419 / 3975

In [ ]:
df = df[(df.height > 0) & (df.height <= 20)]

In [ ]:
df = df[(df.height >= 3) & (df.height <= 20) & (df.n_points >= 500)]

In [ ]:
len(df)

In [ ]:
df = df[df.n_points >= 1000]

### Height distribution plot

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import numpy as np

#heights = [x[3] for x in locations]
#counts = [x[4] for x in locations]
heights = df.height.values
counts = df.n_points.values

fig = plt.figure(figsize=(10,2))
scat = plt.scatter(heights, np.random.uniform(size=(len(heights,))), 
                   c=counts, cmap='Blues', vmin=0, vmax=2500,
                   marker='.', alpha=0.8, edgecolors='none')
ax = plt.gca()
ax.yaxis.set_visible(False)
ax.xaxis.set_ticks(range(0, 21, 2))
ax.set_title('Distribution of pole height (m)')
#ax.legend(*scat.legend_elements())
#plt.clim(0, 2000)
cbar = plt.colorbar(scat, extend="max")
cbar.ax.set_ylabel('#points', rotation=270, labelpad=12)
plt.show()

In [ ]:
plt.savefig('height_dist_500_rgbi.png')

### Location plot

In [ ]:
df = df[(df.height >= 3) & (df.n_points >= 500)]

In [ ]:
import matplotlib

def numfmt(x, pos): # your custom formatter function: divide by 100.0
    s = f'{x / 1000:.0f}'
    return s

import matplotlib.ticker as tkr     # has classes for tick-locating and -formatting
fmt = tkr.FuncFormatter(numfmt)    # create your custom formatter function

cmap = matplotlib.colors.LinearSegmentedColormap.from_list('custom greys', [(.8,.8,.8),(0,0,0)], N=64)

fig = plt.figure(figsize=(10,8))
scat = plt.scatter(df.rd_x, df.rd_y, c ='black', 
                   marker='.', alpha=0.8, edgecolors='none')
ax = plt.gca()
ax.set_title('Located street lights in Weesp')
ax.set_xlabel('RD X (km)')
ax.set_ylabel('RD Y (km)')
ax.xaxis.set_major_formatter(fmt)
ax.yaxis.set_major_formatter(fmt)
ax.set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
plt.savefig('weesp_streetlights_3m_rgbi.png')

## Dataset map plot

In [ ]:
from src.utils import plot_utils
tile_folder = '../../datasets/Weesp/pred_rgbi/'
train_folder = '../../datasets/Weesp_train/all/'

tiles = las_utils.get_tilecodes_from_folder(tile_folder)
train_tiles = las_utils.get_tilecodes_from_folder(train_folder)

tiles_map = plot_utils.plot_tiles_map(tiles, train_tiles, zoom_control=False, opacity=0.25)
tiles_map

In [ ]:
plot_utils.save_tiles_map(tiles_map, 'weesp_tiles.png')

In [ ]:
tiles_map.save('weesp_tiles.html')

## Create LAS with extracted poles

In [ ]:
df = pd.read_csv('lamp_posts_weesp_test.csv')

In [ ]:
tilecode = '2623_9621'
locations_df = df[df.tilecode == tilecode]
locations_df

In [ ]:
# locations = locations_df[['rd_x','rd_y','z','height']].values
locations = [[(x, y, z), (x2, y2, z2)] for x, y, z, x2, y2, z2 in locations_df.iloc[:,0:6].values]
target_label = Labels.STREET_LIGHT

In [ ]:
las_utils.create_pole_las(f'dummy_{tilecode}.laz', locations, target_label)

In [ ]:
df = pd.read_csv('lamp_posts_weesp_test.csv')
target_label = Labels.STREET_LIGHT
for tilecode in df.tilecode:
    locations_df = df[df.tilecode == tilecode]
    locations = [[(x, y, z), (x2, y2, z2)] for x, y, z, x2, y2, z2 in locations_df.iloc[:,0:6].values]
    las_utils.create_pole_las(f'lampposts_{tilecode}.laz', locations, target_label)

## Merge predictions into LAS files

In [ ]:
cloud_folder = '../../datasets/Weesp/las_processor_bundled_out/'
pred_folder = '../../datasets/Weesp/pred_rgbi/'
out_folder = '../../datasets/Weesp/predicted_rgbi/'
cloud_prefix = 'filtered'
pred_prefix = 'filtered'
out_prefix = 'merged'
#label_dict = {Labels.UNLABELLED: Labels.CAR}
label_dict = None

las_utils.merge_cloud_pred_folder(cloud_folder, pred_folder, out_folder,
                                  cloud_prefix=cloud_prefix, pred_prefix=pred_prefix,
                                  out_prefix=out_prefix, label_dict=label_dict,
                                  resume=True)

## Dataset statistics

In [ ]:
import laspy
import pathlib

In [ ]:
folder = '../../datasets/Weesp/predicted_rgbi/'
counts = []
for file in pathlib.Path(folder).glob('*.laz'):
    with laspy.open(file.as_posix()) as f:
        counts.append(f.header.point_count)

In [ ]:
len(counts)

In [ ]:
sum(counts)

## Move LAS files based on number of points

In [ ]:
import glob, os, shutil
import laspy
from lazrs.lazrs import LazrsError

las_folder = '../../datasets/Weesp/las_processor_bundled_out/'
empty_folder = '../../datasets/Weesp/las_processor_bundled_out/empty/'
small_folder = '../../datasets/Weesp/las_processor_bundled_out/10k/'
files = glob.glob(os.path.join(las_folder, '*.laz'))

for i, file in enumerate(files):
    filename = file.split('/')[-1]
    try:
        with laspy.open(file) as fh:
            if fh.header.point_count < 10000:
                print(f'Moving {filename} to 10k [{i}]')
                shutil.move(file, os.path.join(small_folder, filename))
    except LazrsError:
        print(f'Moving {filename} to empty [{i}]')
        shutil.move(file, os.path.join(empty_folder, filename))

In [ ]:
files = glob.glob(os.path.join(las_folder, '*.laz'))
print(f'Total left: {len(files)}')
files = glob.glob(os.path.join(empty_folder, '*.laz'))
print(f'Empty: {len(files)}')
files = glob.glob(os.path.join(small_folder, '*.laz'))
print(f'Smaller than 10k: {len(files)}')

## Visualise individual lamp posts

In [ ]:
df = pd.read_csv('lamp_posts_weesp_rgbi_100_new4.csv', delimiter=',')

In [ ]:
df = df[(df.height >= 3) & (df.n_points >= 500)]

In [ ]:
len(df)

In [ ]:
las_folder = '../../datasets/Weesp/predicted_rgbi/'
las_prefix = 'merged'
target_label = Labels.STREET_LIGHT

In [ ]:
df[df.tilecode=='2641_9611']

In [ ]:
df = df_ext

In [ ]:
idx = np.random.randint(0, len(df))
row_id = df.index[idx]
row_id = 4

obj_df = df.loc[[row_id]]
obj = df.loc[row_id]
tilecode = obj.tilecode
obj_loc = (obj.rd_x, obj.rd_y, obj.z)
obj_top = (obj.tx, obj.ty, obj.tz)
obj_angle = obj.angle

obj_df

In [ ]:
tilecode = '2617_9580'

In [ ]:
cloud = las_utils.read_las(f'{las_folder}/{las_prefix}_{tilecode}.laz')
points = np.vstack((cloud.x, cloud.y, cloud.z)).T
colors = np.vstack((cloud.red, cloud.green, cloud.blue)).T / (2**16 - 1)

In [ ]:
analysis_tools.get_pole_locations(points, cloud.label, cloud.probability, Labels.STREET_LIGHT, Labels.GROUND)

In [ ]:
from src.region_growing import LabelConnectedComp
from src.utils import math_utils
from src.utils import clip_utils

target_idx = np.where(cloud.label == target_label)[0]

noise_components = (LabelConnectedComp(
                        octree_level=8,
                        min_component_size=10)
                    .get_components(points[target_idx]))
noise_filter = noise_components != -1
point_components = (LabelConnectedComp(
                        octree_level=8,
                        min_component_size=100)
                    .get_components(points[target_idx[noise_filter], 0:2]))

cc_labels, counts = np.unique(point_components, return_counts=True)
cc_labels_valid = [l for l in cc_labels if l != -1]

off_eps = 0.05
obj_idx = -1
for cc in cc_labels_valid:
    cc_mask = point_components == cc
    min_x, max_y, max_x, min_y = math_utils.compute_bounding_box(points[target_idx][noise_filter][cc_mask])
    off_h = np.min(points[target_idx][noise_filter][cc_mask, 2]) - obj_loc[2]
    offset = np.sqrt((off_h / np.sin(np.deg2rad(90 - obj_angle)))**2 - off_h**2) + off_eps
    min_x -= offset
    min_y -= offset
    max_x += offset
    max_y += offset
    if min_x <= obj_loc[0] <= max_x and min_y <= obj_loc[1] <= max_y:
        obj_idx = int(cc)
        break
if obj_idx == -1:
    print("No match found!")
    pad = 1
    box = (obj_loc[0]-pad, obj_loc[1]-pad, obj_loc[0]+pad, obj_loc[1]+pad)
else:
    pad = 0.5
    box = (min_x-pad, min_y-pad, max_x+pad, max_y+pad)
    obj_mask = target_idx[noise_filter][cc_mask]

bg_mask = clip_utils.box_clip(points, box, bottom=obj_loc[2]-pad, top=obj_top[2]+2)

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from src.utils import plot_utils

xs = points[bg_mask, 0]
ys = points[bg_mask, 1]
zs = points[bg_mask, 2]
# cs = colors[bg_mask, :]
labels = cloud.label[bg_mask]

estimate = np.vstack((obj_loc, obj_top))

fig = plt.figure(figsize=(10,5))
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)
ax3 = fig.add_subplot(133, projection='3d')

label_set = np.unique(labels)

for label in label_set:
    if label == Labels.NOISE:
        continue
    label_mask = labels == label
    label_str = Labels.get_str(label)
        
    ax1.scatter(xs[label_mask], zs[label_mask], c=plot_utils.cloud_colors[label_str], marker='.', edgecolors='none', alpha=1, label=label_str)
    ax1.plot(estimate[:, 0], estimate[:, 2], c='red', linewidth=3, alpha=0.7, label='Est. pole')

    ax2.scatter(ys[label_mask], zs[label_mask], c=plot_utils.cloud_colors[label_str], marker='.', edgecolors='none', alpha=1)
    ax2.plot(estimate[:, 1], estimate[:, 2], c='red', linewidth=3, alpha=0.7)

    ax3.scatter(xs[label_mask], ys[label_mask], zs[label_mask], c=plot_utils.cloud_colors[label_str], marker='.', edgecolors='none', alpha=0.05)
    ax3.plot(estimate[:, 0], estimate[:, 1], estimate[:, 2], c='red', linewidth=3, alpha=1)

ax1.set_aspect('equal')
ax2.set_aspect('equal')
ax3.set_box_aspect((np.ptp(xs), np.ptp(ys), np.ptp(zs)))
ax3.xaxis.set_ticklabels([])
ax3.yaxis.set_ticklabels([])
ax3.dist = 8

handles, labels = ax1.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(),
           loc='upper center', bbox_to_anchor=(0.5, 1), ncol=int(len(by_label) / 2 + 0.5))

fig.subplots_adjust(wspace=0, hspace=0)

plt.show()

In [ ]:
plt.close('all')

## Sandbox

In [ ]:
import laspy
from src.utils import clip_utils

HEADERS = ['rd_x', 'rd_y', 'z', 'tx', 'ty', 'tz', 'height', 'angle', 'prob', 'n_points', 'in_bld', 'debug']

# las = laspy.read('../datasets/pointcloud/processed_2397_9705.laz')

# tilecode = '2621_9612'
# tilecode = '2622_9593'
tilecode = '2633_9612'
las = laspy.read(f'../../datasets/Weesp/predicted_rgbi/merged_{tilecode}.laz')


points = np.vstack((las.x, las.y, las.z)).T
labels = las.label
# probabilities = las.probability
probabilities = np.zeros((len(labels,)))

In [ ]:
poles = analysis_tools.get_pole_locations(points, labels, probabilities,
                                          Labels.STREET_LIGHT, Labels.GROUND,
                                          octree_level=7)
df = pd.DataFrame(poles, columns=HEADERS)
df[df.height > 3]

In [ ]:
len(df)

In [ ]:
pole_id = 15

location = df.loc[pole_id, ['rd_x', 'rd_y', 'z']].values
top = df.loc[pole_id, ['tx', 'ty', 'tz']].values
z_ground = df.loc[pole_id, 'z']

clip_mask = clip_utils.circle_clip(points, (location[0], location[1]), 2.)
label_mask = labels == Labels.STREET_LIGHT
light = points[label_mask & clip_mask]
# light_sign = points[clip_mask & ((labels == Labels.TRAFFIC_SIGN) | label_mask)]
# light = light_sign

In [ ]:
offset = np.max(np.abs(location - top)[0:2])
ctr_mask = clip_utils.cylinder_clip(points, (location[0], location[1]), offset + 0.1, bottom=z_ground+0.1, top=top[2])
# light = points[ctr_mask | (label_mask & clip_mask)]
new_light = points[ctr_mask]
new_pole = analysis_tools.extract_pole(new_light, z_ground, step=0.1)[0]

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

xs = light[:, 0]
ys = light[:, 1]
zs = light[:, 2]

line = np.vstack((location, top))
line2 = np.vstack((new_pole[0:3], new_pole[3:6]))

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs, ys, zs, alpha=0.05, marker='.')
ax.plot(line[:, 0], line[:, 1], line[:, 2], 'r', linewidth=3)
ax.plot(line2[:, 0], line2[:, 1], line2[:, 2], 'orange', linewidth=3)
# ax.plot(line3[:, 0], line3[:, 1], line3[:, 2], 'orange', linewidth=3)
ax.set_box_aspect((np.ptp(xs), np.ptp(ys), np.ptp(zs)))

plt.show()

In [ ]:
from sklearn.decomposition import PCA
import src.utils.math_utils as math_utils

def get_xyzr(points, z, margin):
    clip_mask = (points[:, 2] >= z - margin) & (points[:, 2] < z + margin)
    if np.count_nonzero(clip_mask) > 0:
        x_mean = np.mean(points[clip_mask, 0])
        y_mean = np.mean(points[clip_mask, 1])
        x_r = (np.max(points[clip_mask, 0]) - np.min(points[clip_mask, 0])) / 2.
        y_r = (np.max(points[clip_mask, 0]) - np.min(points[clip_mask, 0])) / 2.
        r = x_r if x_r > y_r else y_r
        return x_mean, y_mean, z, r
    else:
        return np.nan, np.nan, z, np.nan


def extract_pole(points, ground_est=None, step=0.1):
    debug = 0
    z_min = np.min(points[:, 2])
    z_max = np.max(points[:, 2])
    if ground_est is None:
        ground_est = z_min
    xyzr = np.array([[*get_xyzr(points, z, step)]
                     for z in np.arange(z_min + step, z_max, 2*step)])
    valid_mask = xyzr[:, 3] <= 0.15
    if np.count_nonzero(valid_mask) == 0:
        logger.debug('Not enough data to extract pole.')
        debug = 4
        origin = np.mean(points, axis=0)
        direction_vector = np.array([0, 0, 1])
    elif np.count_nonzero(valid_mask) == 1:
        logger.debug('Not enough data to determine slope.')
        debug = 3
        origin = xyzr[valid_mask, 0:3][0]
        direction_vector = np.array([0, 0, 1])
    else:
        pca = PCA(n_components=1).fit(xyzr[valid_mask, 0:3])
        origin = pca.mean_
        direction_vector = pca.components_[0]
        if direction_vector[2] < 0:
            direction_vector *= -1
    extent = (origin[2] - ground_est, z_max - origin[2])
    multiplier = np.sum(np.linalg.norm(direction_vector, 2))
    x, y, z = origin - direction_vector * extent[0] * multiplier
    x2, y2, z2 = origin + direction_vector * extent[1] * multiplier
    height = np.sum(extent) * multiplier
    angle = math_utils.vector_angle(direction_vector)
    return (x, y, z, x2, y2, z2, height, angle), debug

In [ ]:
z_min = np.min(light[:,2])
z_max = np.max(light[:,2])
step = 0.1

xyzstd = np.array([[*analysis_tools.get_xystd(light, z, step)]
                   for z in np.arange(z_min + step, z_max, 2*step)])
xyzr = np.array([[*get_xyzr(light, z, step)]
                 for z in np.arange(z_min + step, z_max, 2*step)])

In [ ]:
pd.DataFrame(xyzr)

In [ ]:
np.nanpercentile(xyzstd[:, 3], 20)

In [ ]:
valid_mask = xyzstd[:, 3] <= np.nanpercentile(xyzstd[:, 3], 20)
np.count_nonzero(valid_mask)

In [ ]:
pd.DataFrame(xyzstd[valid_mask])

In [ ]:
valid_mask = xyzr[:, 3] <= 0.20
pd.DataFrame(xyzr[valid_mask])

In [ ]:
sub_ply_file = 'processed_2601_9615.npz'
data = np.load(sub_ply_file)
sub_colors = np.vstack((data['red'], data['green'], data['blue'],
                        data['intensity'])).T
sub_labels = data['label'].squeeze()

In [ ]:
sub_labels.shape

In [ ]:
sub_colors.shape

In [ ]:
data['label']

In [ ]:
red = (data['red'].squeeze() * 255).astype(np.uint8)
green = (data['green'].squeeze() * 255).astype(np.uint8)
blue = (data['blue'].squeeze() * 255).astype(np.uint8)
intensity = (data['intensity'].squeeze() * 255).astype(np.uint8)
label = data['label'].squeeze().astype(np.uint8)

In [ ]:
np.savez('test.npz', red=red, green=green, blue=blue, intensity=intensity, label=label)